In [1]:
import pandas as pd
import geopandas as gpd
%matplotlib inline

import warnings
warnings.simplefilter(action="ignore")

import cityImage as ci

In [2]:
# initialise path, names, etc.
city_name = 'UPB'
epsg = 4326
crs = 'EPSG:'+str(epsg)

### Download the network

Choose between the following methods:
* `OSMplace`, provide an OSM place name (e.g. City).
* `polygon`, provide an WGS polygon of the case-study area.
* `distance_from_address`, provide a precise address and define parameter `distance` (which is otherwise not necessary)

In [3]:
# download graph and clean (see network notebook for details on the cleaning process)

#place = 'University of the Philippines Baguio, Baguio City, Philippines'
#download_method = 'distance_from_address'
#distance = 500
#nodes_graph, edges_graph = ci.get_network_fromOSM(place, download_method, 'walk', epsg, distance = distance)
#nodes_graph, edges_graph = ci.clean_network(nodes_graph, edges_graph, dead_ends = True, remove_disconnected_islands = True,
#                            self_loops = True, same_uv_edges = True)

### Load from local path

In [4]:
loading_path = 'Outputs/'+city_name+'/'
nodes_graph = gpd.read_file(loading_path+'nodes.shp')
edges_graph = gpd.read_file(loading_path+'edges.shp')
barriers = gpd.read_file(loading_path+city_name+'_barriers.shp')
try:
    nodes_graph, edges_graph = nodes_graph.to_crs(epsg=epsg), edges_graph.to_crs(epsg=epsg)
except:
    nodes_graph.crs, edges_graph.crs = crs, crs

nodes_graph.index, edges_graph.index  = nodes_graph.nodeID, edges_graph.edgeID
nodes_graph.index.name, edges_graph.index.name  = None, None

In [5]:
#fig = ci.plot_gdf(edges_graph, scheme = None, black_background = False, figsize = 15, alpha = 1.0, color = 'black',
#            title = city_name+': Street Network')

## Barriers identification

Choose between the following methods:
* *OSMplace*, provide an OSM place name (e.g. City).
* *distance_from_address*, provide a precise address and define parameter "distance" (which is otherwise not necessary)
* *polygon*, provide a Polygon (coordinates must be in units of latitude-longitude degrees).

In [6]:
# define method and create envelope with wgs coordinate system
download_method = 'polygon'
place = ci.envelope_wgs(edges_graph)
distance = None

In [7]:
road_barriers = gpd.GeoDataFrame()
secondary_road = gpd.GeoDataFrame() # optional - only in relation to walkability
water_barriers = gpd.GeoDataFrame()
railway_barriers = gpd.GeoDataFrame()
park_barriers = gpd.GeoDataFrame()

In [8]:
#barriers = road_barriers.append(secondary_road)
#barriers = water_barriers.append(barriers)
#barriers = railway_barriers.append(barriers)
#barriers = park_barriers.append(barriers)
barriers.reset_index(inplace = True, drop = True)
barriers['barrierID'] = barriers.index.astype(int)

envelope = edges_graph.unary_union.envelope
barriers_within = barriers[barriers.intersects(envelope)]

### Visualisation

In [9]:
#fig = ci.plot_barriers(barriers_within, black_background = False, fig_size = 15, title = city_name+': Barriers', legend = True)

In [10]:
# save barriers_gdf
saving_path = 'Outputs/'+city_name+'/'
barriers.to_file(saving_path+"barriers.shp", driver='ESRI Shapefile')

### Assigning barriers to street segments

### Type of barriers

Choose between the following methods:
* *Positive barriers* - pedestrian perspective: Waterbodies, Parks.
* *Negative barriers* - pedestrian perspective: Major Roads, Railway Structures.
* *Structuring barriers* - barriers which structure and shape the image of the city: Waterbodies, Major roads, Railways.

In [11]:
sindex = edges_graph.sindex

# along and within POSITIVE BARRIERS
# rivers
edges_graph = ci.along_water(edges_graph, barriers_within)
# parks
#edges_graph = ci.along_within_parks(edges_graph, barriers_within)

edges_graph = edges_graph.assign(a_rivers='[]').astype({'a_rivers': str})
edges_graph = edges_graph.assign(w_parks='[]').astype({'w_parks': str})

# altogheter
edges_graph['p_barr'] = edges_graph['a_rivers']+edges_graph['w_parks']
#edges_graph['p_barr'] = edges_graph.apply(lambda row: list(set(row['p_barr'])), axis = 1)

#edges_graph = gpd.GeoDataFrame(columns=['p_barr', 'n_barr', 'a_rivers', 'w_parks'])
# along NEGATIVE BARRIERS                                    
tmp = barriers_within[barriers_within['type'].isin(['railway', 'road', 'secondary_road'])]
edges_graph['n_barr'] = edges_graph.apply(lambda row: ci.barriers_along(row['edgeID'], edges_graph, tmp, sindex,
                                           offset = 25), axis = 1)

# crossing any kind of barrier but parks - STRUCTURING BARRIERS
edges_graph = ci.assign_structuring_barriers(edges_graph, barriers_within)

## Visualisation

In [12]:
# positive barriers
edges_graph = edges_graph.assign(p_bool=0).astype({'p_bool': int})
edges_graph['p_bool'] = edges_graph.apply(lambda row: True if len(row['p_barr']) > 0 else False, axis = 1)
tmp = edges_graph[edges_graph.p_bool == True].copy()
#fig = ci.plot_gdf(tmp, black_background = False, figsize = 15, color = 'red', title = city_name+': Streets along parks and rivers', 
#              legend = False, base_map_gdf = edges_graph, base_map_alpha = 0.3, base_map_color = 'black')

In [13]:
# negative barriers
edges_graph = edges_graph.assign(n_bool=0).astype({'n_bool': int})
edges_graph['n_bool'] = edges_graph.apply(lambda row: True if len(row['n_barr']) > 0 else False, axis = 1)
tmp = edges_graph[edges_graph.n_bool == True].copy()
#fig = ci.plot_gdf(tmp, black_background = False, figsize = 15, color = 'red', title = city_name+': Streets along negative barriers',
#              legend = False, base_map_gdf = edges_graph, base_map_alpha = 0.3, base_map_color = 'black')

### Save 

In [14]:
# convertin list fields to string
#to_convert = ['a_rivers', 'w_parks','n_barr', 'p_barr']
edges_graph_string = edges_graph.copy()
#for column in to_convert: 
#    edges_graph_string[column] = edges_graph_string[column].astype(str)

saving_path = 'Outputs/'+city_name+'/'
edges_graph_string.to_file(saving_path+"edges.shp", driver='ESRI Shapefile')
nodes_graph.to_file(saving_path+'nodes.shp', driver='ESRI Shapefile')